<a href="https://colab.research.google.com/github/diogenesjusto/FIAP-On-Line-Graduation/blob/main/code/11_Ensemble_Learning_Bagging_e_Boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# carga de bibliotecas
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# carga de dados
df = pd.read_csv("https://raw.githubusercontent.com/diogenesjusto/FIAP-On-Line-Graduation/main/datasets/arvore_titanic.csv")

# transformações de dados
du_Sex = pd.get_dummies(df['Sex'])
du_Pclass = pd.get_dummies(df['Pclass'], prefix='class')
dtAge = df['Age'].fillna(df.groupby('Pclass')['Fare'].transform('mean'))

df = df.join(du_Sex)
df = df.join(du_Pclass)
df['AgeFilledNa'] = dtAge

In [3]:
# Comparativo de execução do Random Forest com e sem parâmetro de bagging
for i in range(10):
  x_treino, x_teste, y_treino, y_teste = train_test_split(df[['female', 'class_1','class_2', 'SibSp', 'Parch', 'AgeFilledNa']]
                                                          , df['Survived'],test_size=0.3266, random_state=i)

  # Bootstrap Ativo
  mod = RandomForestClassifier(n_estimators = 500, random_state = 1, bootstrap=True)
  mod = mod.fit(x_treino, y_treino)
  y_prev = mod.predict(x_teste)
  accuracy_bootstrap_true = accuracy_score(y_teste, y_prev)

  # Bootstrap Desativado
  mod = RandomForestClassifier(n_estimators = 500, random_state = 1, bootstrap=False)
  mod = mod.fit(x_treino, y_treino)
  y_prev = mod.predict(x_teste)
  accuracy_bootstrap_false = accuracy_score(y_teste, y_prev)


  print(accuracy_bootstrap_true, accuracy_bootstrap_false, accuracy_bootstrap_true-accuracy_bootstrap_false)

0.791095890410959 0.7876712328767124 0.003424657534246589
0.7671232876712328 0.7465753424657534 0.020547945205479423
0.8013698630136986 0.7671232876712328 0.03424657534246578
0.8047945205479452 0.8013698630136986 0.003424657534246589
0.815068493150685 0.8116438356164384 0.003424657534246589
0.8116438356164384 0.8047945205479452 0.006849315068493178
0.8116438356164384 0.8184931506849316 -0.006849315068493178
0.7876712328767124 0.7808219178082192 0.006849315068493178
0.821917808219178 0.8116438356164384 0.010273972602739656
0.8184931506849316 0.8116438356164384 0.006849315068493178


In [5]:
!pip install catboost

from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
import catboost as ctb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [25]:
from catboost.core import CatBoostClassifier
import time 

classifiers = [
    GradientBoostingClassifier(),
    XGBClassifier(),
    lgb.LGBMClassifier(),
    ctb.CatBoostClassifier()
]
classifier_names = ['GBC', 'XGBoost', 'LightGBM', 'CatBoost']

x_treino, x_teste, y_treino, y_teste = train_test_split(df[['female', 'class_1','class_2', 'SibSp', 'Parch', 'AgeFilledNa']]
                                                          , df['Survived'],test_size=0.3266, random_state=1)

for idx, clf in enumerate(classifiers):
  inicio = time.time()
  
  if classifier_names[idx]=='CatBoost':
    mod = clf.fit(x_treino, y_treino, silent= True)
  else:
    mod = clf.fit(x_treino, y_treino)

  y_prev = mod.predict(x_teste)
  acc = accuracy_score(y_teste, y_prev)
  fim = time.time()

  print(classifier_names[idx], '| Acc=',acc,'| Tempo para execução:', fim-inicio, 'segundos')


GBC | Acc= 0.7636986301369864 | Tempo para execução: 0.08865189552307129 segundos
XGBoost | Acc= 0.7705479452054794 | Tempo para execução: 0.05170583724975586 segundos
LightGBM | Acc= 0.7808219178082192 | Tempo para execução: 0.040944576263427734 segundos
CatBoost | Acc= 0.7636986301369864 | Tempo para execução: 0.5190300941467285 segundos


In [29]:
# 2a tentativa: aumento da base de treino em 10x
from catboost.core import CatBoostClassifier
import time 

classifiers = [
    GradientBoostingClassifier(),
    XGBClassifier(),
    lgb.LGBMClassifier(),
    ctb.CatBoostClassifier()
]
classifier_names = ['GBC', 'XGBoost', 'LightGBM', 'CatBoost']

x_treino, x_teste, y_treino, y_teste = train_test_split(df[['female', 'class_1','class_2', 'SibSp', 'Parch', 'AgeFilledNa']]
                                                          , df['Survived'],test_size=0.3266, random_state=1)

# Replicando base de treino para medição de tempo de execução de algoritmos
for i in range(6):
  x_treino = pd.concat([x_treino, x_treino])
  y_treino = pd.concat([y_treino, y_treino])

print('Tamanho da base de treino:', y_treino.shape)

for idx, clf in enumerate(classifiers):
  inicio = time.time()
  
  if classifier_names[idx]=='CatBoost':
    mod = clf.fit(x_treino, y_treino, silent= True)
  else:
    mod = clf.fit(x_treino, y_treino)

  y_prev = mod.predict(x_teste)
  acc = accuracy_score(y_teste, y_prev)
  fim = time.time()

  print(classifier_names[idx], '| Acc=',acc,'| Tempo para execução:', fim-inicio, 'segundos')

Tamanho da base de treino: (38336,)
GBC | Acc= 0.7636986301369864 | Tempo para execução: 1.3645613193511963 segundos
XGBoost | Acc= 0.7705479452054794 | Tempo para execução: 1.254347801208496 segundos
LightGBM | Acc= 0.7602739726027398 | Tempo para execução: 0.2991676330566406 segundos
CatBoost | Acc= 0.7328767123287672 | Tempo para execução: 6.484435558319092 segundos


In [32]:
from catboost.core import CatBoostClassifier
import time 

classifiers = [
    GradientBoostingClassifier(),
    XGBClassifier(),
    lgb.LGBMClassifier(),
    ctb.CatBoostClassifier()
]
classifier_names = ['GBC', 'XGBoost', 'LightGBM', 'CatBoost']

for iRandomState in range(5):

  x_treino, x_teste, y_treino, y_teste = train_test_split(df[['female', 'class_1','class_2', 'SibSp', 'Parch', 'AgeFilledNa']]
                                                            , df['Survived'],test_size=0.3266, random_state=iRandomState)

  # Replicando base de treino para medição de tempo de execução de algoritmos
  for i in range(6):
    x_treino = pd.concat([x_treino, x_treino])
    y_treino = pd.concat([y_treino, y_treino])

  for idx, clf in enumerate(classifiers):
    inicio = time.time()
    
    if classifier_names[idx]=='CatBoost':
      clf = CatBoostClassifier(iterations=20, learning_rate=0.15,depth=10)
      mod = clf.fit(x_treino, y_treino, silent= True)
    else:
      mod = clf.fit(x_treino, y_treino)

    y_prev = mod.predict(x_teste)
    acc = accuracy_score(y_teste, y_prev)
    fim = time.time()

    print(classifier_names[idx], '| Acc=',acc,'| Tempo para execução:', fim-inicio, 'segundos')

GBC | Acc= 0.8116438356164384 | Tempo para execução: 1.3730709552764893 segundos
XGBoost | Acc= 0.8047945205479452 | Tempo para execução: 1.2530746459960938 segundos
LightGBM | Acc= 0.7773972602739726 | Tempo para execução: 0.2980766296386719 segundos
CatBoost | Acc= 0.8184931506849316 | Tempo para execução: 0.16159725189208984 segundos
GBC | Acc= 0.7602739726027398 | Tempo para execução: 1.3432857990264893 segundos
XGBoost | Acc= 0.7705479452054794 | Tempo para execução: 1.25520658493042 segundos
LightGBM | Acc= 0.7602739726027398 | Tempo para execução: 0.29300403594970703 segundos
CatBoost | Acc= 0.7705479452054794 | Tempo para execução: 0.15827441215515137 segundos
GBC | Acc= 0.797945205479452 | Tempo para execução: 1.363389492034912 segundos
XGBoost | Acc= 0.8013698630136986 | Tempo para execução: 1.2402095794677734 segundos
LightGBM | Acc= 0.773972602739726 | Tempo para execução: 0.29149818420410156 segundos
CatBoost | Acc= 0.7945205479452054 | Tempo para execução: 0.1629915237426